# 对象池 `ObjectPool.hpp`

当需要频繁地创建和销毁同一种类型的对象时，向操作系统申请和释放内存是一个相对较慢的操作

**对象池**：预先分配一大块内存，并在这块内存上重复利用对象，从而避免了频繁的系统内存调用

用法：假设有一个经常需要创建和销毁的类 A
- 创建专门用于管理 A 对象的池子 `ObjectPool<Order> APool;`
    ```cpp
    ObjectPool() :_pool(sizeof(T)) {}  // 初始化内存池，块大小为 T 类型的大小
    ```
- 获取对象 `A* myA1 = APool.construct();`
    ```cpp
    T* construct()
    {
        void * mem = _pool.malloc();  // 从内存池中分配内存块（注意这里是 boost::pool<> 类型的 malloc，不需要向操作系统申请内存）
        if (!mem)  // 检查内存分配是否成功
            return nullptr;  // 分配失败时返回nullptr

        T* pobj = new(mem) T();  // 使用placement new在指定内存上构造对象
        return pobj;  // 返回构造好的对象指针
    }
    ```
- 归还对象 `APool.destroy(myA1);`
    ```cpp
    void destroy(T* pobj)
    {
        pobj->~T();  // 调用对象的析构函数，清理对象资源
        _pool.free(pobj);  // 将内存归还给内存池（同样这里是 boost::pool<> 类型的 free）
    }
    ```
- 释放池子中未使用的内存块 `APool.release();`
    ```cpp
    void release()
    {
        _pool.release_memory();  // 释放内存池中未使用的内存块
    }
    ```

## 源码
```cpp
/**
 * @class ObjectPool
 * @brief 对象池模板类
 * @tparam T 对象类型模板参数
 * 
 * 该类提供了基于Boost.Pool的高性能对象池功能，
 * 支持任意类型对象的快速分配和回收。
 * 通过内存池技术减少内存碎片，提高分配效率。
 */
template < typename T>
class ObjectPool
{
	boost::pool<> _pool;  // Boost内存池对象，管理指定大小的内存块

public:
	/**
	 * @brief 默认构造函数
	 * 
	 * 初始化对象池，设置内存块大小为模板类型T的大小。
	 * 内存池会自动管理内存的分配和回收。
	 */
	ObjectPool() :_pool(sizeof(T)) {}  // 初始化内存池，块大小为T类型的大小

	/**
	 * @brief 虚析构函数
	 * 
	 * 虚析构函数确保继承类能够正确析构。
	 * Boost.Pool会自动管理内存的释放。
	 */
	virtual ~ObjectPool() {}  // 虚析构函数，支持继承

	/**
	 * @brief 构造并返回一个新对象
	 * @return T* 成功时返回对象指针，失败时返回nullptr
	 * 
	 * 该函数从内存池中分配内存，然后使用placement new构造对象。
	 * 如果内存分配失败，返回nullptr。
	 * 
	 * 注意：返回的对象需要手动调用destroy函数进行销毁。
	 */
	T* construct()
	{
		void * mem = _pool.malloc();  // 从内存池中分配内存块
		if (!mem)  // 检查内存分配是否成功
			return nullptr;  // 分配失败时返回nullptr

		T* pobj = new(mem) T();  // 使用placement new在指定内存上构造对象
		return pobj;  // 返回构造好的对象指针
	}

	/**
	 * @brief 销毁对象并回收内存
	 * @param pobj 要销毁的对象指针
	 * 
	 * 该函数先调用对象的析构函数，然后将内存归还给内存池。
	 * 使用RAII原则确保资源的正确释放。
	 * 
	 * 注意：该函数会调用对象的析构函数，确保对象正确清理。
	 */
	void destroy(T* pobj)
	{
		pobj->~T();  // 调用对象的析构函数，清理对象资源
		_pool.free(pobj);  // 将内存归还给内存池，供后续使用
	}

	/**
	 * @brief 手动释放未使用的内存
	 * 
	 * 该函数释放内存池中未使用的内存块，减少内存占用。
	 * 适用于需要优化内存使用的场景。
	 * 
	 * 注意：释放后，已分配的对象仍然有效，但新分配可能触发新的内存分配。
	 */
	void release()
	{
		_pool.release_memory();  // 释放内存池中未使用的内存块
	}
};
```

# 自旋锁 SpinMutex.hpp
普通互斥锁的问题：
- 当普通互斥锁的占用时间比较短（例如几十纳秒），那么 *阻塞* 和 *解除阻塞* 相较而言会非常耗时（线程的上下文切换，需要操作系统内核介入，可能达到几千纳秒）

自旋锁的设计理念：
- 提供一个标志位，当这个标志位为真（被占用），一直循环检测是否为假（已经解除占用），所以几乎可以以零延迟获得/解除该标志位
- 所以只适用于标志位占用时间较短的情况，如果比较长，就会有较长时间的 *CPU空转*

该文件包含两个类：**自旋锁 `SpinMutex`** 和 **自旋锁包装器 `SpinLock`**

使用方式：
- 定义一个自旋锁 
    ```cpp
    SpinMutex counter_mutex;
    ```
- 在需要访问共享资源的多个代码块中，创建各自的自旋锁包装器，并用其包装创建的自旋锁（遵循RAII，即创建加锁，销毁解锁）
    ```cpp
    {
        SpinLock lock(counter_mutex); // 创建时，自动加锁
    } // 离开作用域时，自动解锁
    ```

## 内存屏障
对于单个线程，编译器为了提升执行速度，可能会打乱原子级的代码顺序。当有多个线程时，上述操作可能会导致问题。

当对某线程中的某原子操作指定下述模式时
- `std::memory_order_relaxed`：不设置任何屏障，即任由编译器调整底层执行顺序
- `std::memory_order_release`：向下屏障，上面的任何内存操作不能挪到该操作的下面
- `std::memory_order_acquire`：向上屏障，下面的任何内存操作不能挪到该操作的上面

## 自旋锁 `SpinMutex`
```cpp
/**
 * @class SpinMutex
 * @brief 自旋互斥量类
 * 
 * 该类实现了基于原子操作的自旋互斥量，通过自旋等待实现线程同步。
 * 适用于锁持有时间很短的场景，避免了传统互斥量的线程阻塞开销。
 * 使用内存序控制确保正确的内存同步和可见性。
 */
class SpinMutex
{
private:
	std::atomic<bool> flag = { false };  // 原子布尔标志，false表示未锁定，true表示已锁定

public:
	/**
	 * @brief 获取锁
	 * 
	 * 该函数通过自旋等待获取锁。如果锁已被占用，会持续自旋等待直到获得锁。
	 * 使用exchange操作确保原子性，通过内存序控制保证正确的内存同步。
	 * 
	 * 注意：该函数会一直自旋直到获得锁，适用于锁持有时间很短的场景。
	 */
	void lock()
	{
		for (;;)  // 无限循环，直到成功获得锁
		{
			if (!flag.exchange(true, std::memory_order_acquire))  // 尝试将标志设置为true，如果原值为false则成功
				break;  // 成功获得锁，跳出循环

			while (flag.load(std::memory_order_relaxed))  // 等待锁被释放，使用relaxed内存序减少开销
			{
#ifdef _MSC_VER  // Windows平台
				_mm_pause();  // 使用Intel SSE指令暂停CPU，优化自旋等待
#else  // Linux/Unix平台
				__builtin_ia32_pause();  // 使用GCC内置函数暂停CPU，优化自旋等待
#endif
			}
		}
	}

	/**
	 * @brief 释放锁
	 * 
	 * 该函数释放当前持有的锁，允许其他线程获取锁。
	 * 使用release内存序确保锁释放操作对其他线程可见。
	 */
	void unlock()
	{
		flag.store(false, std::memory_order_release);  // 将标志设置为false，表示锁已释放
	}
};
```

## 自旋锁包装器 `SpinLock`
```cpp
/**
 * @class SpinLock
 * @brief 自旋锁RAII包装器类
 * 
 * 该类提供了RAII风格的自旋锁管理，在构造时自动获取锁，
 * 在析构时自动释放锁，确保锁的正确管理。
 * 禁止拷贝构造和赋值操作，防止意外的锁管理问题。
 */
class SpinLock
{
public:
	/**
	 * @brief 构造函数，自动获取锁
	 * @param mtx 要管理的自旋互斥量引用
	 * 
	 * 在构造时自动调用mutex的lock函数获取锁。
	 * 使用引用确保对同一个互斥量进行操作。
	 */
	SpinLock(SpinMutex& mtx) :_mutex(mtx) { _mutex.lock(); }  // 构造时自动获取锁

	/**
	 * @brief 删除拷贝构造函数
	 * 
	 * 禁止拷贝构造，防止多个SpinLock对象管理同一个锁，
	 * 避免重复加锁或提前释放锁的问题。
	 */
	SpinLock(const SpinLock&) = delete;  // 删除拷贝构造函数

	/**
	 * @brief 删除赋值操作符
	 * 
	 * 禁止赋值操作，防止多个SpinLock对象管理同一个锁，
	 * 确保锁管理的唯一性和安全性。
	 */
	SpinLock& operator=(const SpinLock&) = delete;  // 删除赋值操作符

	/**
	 * @brief 析构函数，自动释放锁
	 * 
	 * 在析构时自动调用mutex的unlock函数释放锁。
	 * 确保即使发生异常，锁也能被正确释放。
	 */
	~SpinLock() { _mutex.unlock(); }  // 析构时自动释放锁

private:
	SpinMutex&	_mutex;  // 自旋互斥量的引用，用于锁的管理
};
```